# Computing transport coefficients with Molly

ANR SINEQ Summer School - 29/09/23

**Author: Renato Spacek**

In this practical session, we will discuss how to compute transport coefficients in [Molly.jl](https://github.com/JuliaMolSim/Molly.jl), a molecular dynamics package written in Julia. In particular, the example we consider is the computation of shear viscosity using the transverse force field method ([Joubaud & Stoltz, 2012](https://epubs.siam.org/doi/abs/10.1137/110836237)).

To simulate the shearing effect, we consider a sinusoidal force profile which applies, to each particle, a force in the $x$-direction depending on its $y$-coordinate. More precisely, it is given by

<a id="sin_fp"></a>
$$
    \tag{1}
    f_y(u) = \sin\left(\frac{2\pi u}{L_y}\right),
$$

with $L_y$ the boundary length in the $y$-direction. The effect of the forcing is illustrated below.

<p align="center">
    <img src="../assets/sin_visc3.gif" controls title="Title"></img>
</p>

For a system of $N$ particles with boundary length $L$, the shear viscosity $\sigma$ can be computed as

<a id="sv"></a>
$$
    \tag{2}
    \sigma=\rho\left(\frac{1}{2\rho_1} - \gamma\right)\left(\frac{L}{2\pi}\right)^2,
$$

where $\rho = N/L^3$ is the density of the system, $\gamma$ is the friction, and $\rho_1$ is relevant linear response, which can be computed with two approaches, both of which are discussed here: (i) the nonequilibrium method; and (ii) the Green-Kubo method.

For both approaches, the dynamics we consider is Langevin dynamics, which evolves the positions $q$ and momenta $p$ according to the SDE

<a id="lang"></a>
$$
\tag{3}
\begin{align*}
    \begin{cases}
        dq_t = M^{-1}p_t \, dt, \\
        dp_t = -\nabla V(q_t) \, dt - \gamma M^{-1}p_t \, dt+\sqrt{\dfrac{2\gamma}{\beta}} dW_t.
    \end{cases}
\end{align*}
$$

We briefly discuss the definition of shear viscosity and the derivation of the expression <a href="#sv">(2)</a> in the next section.


## Definition of shear viscosity

The shear viscosity $\sigma$ is defined by the following ODE

<a id="sv_ode"></a>
$$
    \tag{4}
    -\sigma u''_x(Y) + \gamma\rho u_x(Y) = \rho f_y(Y),
$$

whose solutions are periodic, thus the magnitude of the linear response can be estimated from its Fourier coefficients. We write

<a id="ckh"></a>
$$
    \tag{5}
    c_k(h) = \frac{1}{L}\int_0^L h(s)\exp\left(\frac{2ik\pi s}{L}\right) ds
$$

as the $k$-th Fourier coefficient for an $L$-periodic function $h$. Applying <a href="#ckh">(5)</a> to <a href="#sv_ode">(4)</a> yields an expression for the shear viscosity:

$$
    \sigma=\rho\left(\frac{c_k(f_y)}{c_k(u_x)} - \gamma\right)\left(\frac{L}{2k\pi}\right)^2.
$$

The Fourier coefficient $c_k(u_x)$ can be estimated directly from trajectory averages. Thus, we define the response observable as the empirical Fourier coefficients:


<a id="Rk"></a>
$$
    \tag{6}
    R_k(q,p) = \frac{1}{N}\sum_{i=1}^N \frac{p_{i1}}{m}\exp\left(\frac{2ik\pi q_{i2}}{L}\right).
$$

In practice, it is sufficient to to consider $k=1$, i.e. the first Fourier coefficient. Furthermore, by analytically computing the Fourier coefficient of the forcing profile, we obtain the expression <a href="#sv">(2)</a>.

For a more precise and detailed discussion, see [Joubaud & Stoltz, 2012](https://epubs.siam.org/doi/abs/10.1137/110836237).


---
## Technique 1: Nonequilibrium method

For the nonequilibrium approach, we consider the case where the Langevin dynamics <a href="#lang">(3)</a> is perturbed by some <span style="color:blue">nongradient force</span> $F$ of magnitude $\eta$:

$$
\begin{align*}
    \begin{cases}
        dq_t = M^{-1}p_t \, dt, \\
        dp_t = -\nabla V(q_t) \, dt \textcolor{blue}{+ \eta F(q_t) \, dt} - \gamma M^{-1}p_t \, dt + \sqrt{\dfrac{2\gamma}{\beta}} dW_t.
    \end{cases}
\end{align*}
$$

As discussed, we consider the case where $F$ corresponds to a shearing force, which applies a force in the $x$-direction which depends on the $y$-coordinate:

$$
    F(q) = \begin{pmatrix} f_y(q_2) \\ 0 \\ 0 \end{pmatrix},
$$

with $f_y$ the sinusoidal forcing profile defined in <a href="#sin_fp">(1)</a>.

**Linear response theory:** The perturbation $\eta F$ is expected to induce a response from the system, expressed as a steady-state average $\mathbb{E}_\eta(R)$ for some observable $R$ of interest. For small values of $\eta$, the response is linear in $\eta$. The *linear response* $\rho_1$ is thus defined as the proportionality constant between the perturbation and the response:

$$
    \rho_1 = \lim_{\eta\to 0} \frac{\mathbb{E}_\eta(R)}{\eta}.
$$

In the case of shear viscosity, the observable $R$ corresponds to the first empirical Fourier coefficient <a href="#Rk">(6)</a>, which reads

<a id="Rk1"></a>
$$
    \tag{7}
    R(q,p) = \frac{1}{N}\sum_{i=1}^N \frac{p_{i1}}{m}\exp\left(\frac{2i\pi q_{i2}}{L}\right).
$$

---
#### Implementation in Molly

We start by importing the necessary packages.

In [ ]:
using Molly
using LinearAlgebra
using Statsbase
using Plots

Recall the overall structure for the `System` object, whose fields must be appropriately defined:

```julia
"""
System(;
        atoms,
        coords,
        boundary,
        velocities=nothing,
        atoms_data=[],
        topology=nothing,
        pairwise_inters=(),
        specific_inter_lists=(),
        general_inters=(),
        constraints=(),
        neighbor_finder=NoNeighborFinder(),
        loggers=(),
        force_units=u"kJ * mol^-1 * nm^-1",
        energy_units=u"kJ * mol^-1",
        k=default_k(energy_units))
"""
```

We define a function `place_atoms_on_3D_lattice()`, which initializes coordinates on a cubic lattice.

In [ ]:
"""
    place_atoms_on_3D_lattice(Nx::Integer, Ny::Integer, Nz::Integer, boundary)
"""
function place_atoms_on_3D_lattice(Nx::Integer, Ny::Integer, Nz::Integer, boundary)
    (Lx, Ly, Lz) = boundary.side_lengths
    reshape([SVector(i*Lx/Nx, j*Ly/Ny, k*Lz/Nz) for i = 0:Nx-1, j = 0:Ny-1, k = 0:Nz-1], Nx*Ny*Nz)
end
;

#### Interaction potential and neighbor finder

For our system, we consider the Lennard-Jones interaction potential:

$$
    V(r) = 4\epsilon\left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right],
$$

where
- $\sigma$ = distance at which $V(\sigma) = 0$;
- $\epsilon$ = depth of potential well.

By applying a cutoff radius $r_c$, the resulting truncated potential $V_\text{tr}(r)$ reads

$$
\begin{align*} V_\text{tr}(r) = 
    \begin{cases}
        V(r) - V(r_c), \quad &r \leq r_c, \\
        0, &r > r_c.
    \end{cases}
\end{align*}
$$

<p align="center">
    <img src="../assets/LJ_plot2.png" alt="drawing" width="600"/>
</p>

#### Simulation parameters and initialization

We now define the simulation parameters, the interaction potential and initialize positions and momenta. In particular, we

- define the Lennard-Jones interacting potential with cutoff, and neighbor finder algorithm;
- define `coords` using the `place_atoms_on_3D()` function defined above;
- define `velocities` with the `random_velocity()` Molly function;
- define `boundary` with the `CubicBoundary()` Molly function;
- create `atoms` acting under the Lennard-Jones potential with the `Atom()` Molly function.

**Remark:** Without loss of generality, we consider a domain with equal side lengths, i.e. $L = L_x = L_y = L_z$.

In [ ]:
Nsteps = 1000 # number of steps
dt = 1e-3 # timestep, check its not too small

Ns = 5 # number of particles "per side"
N = Ns^3 # total number of particles

# Lennard-Jones parameters
ϵ = 1.0 # depth of potential well
σ = 1.0 # distance at which V(σ) = 0
rc = 2.5*σ # interaction cutoff radius
m = 1.0 # mass of particle
kB = 1.0 # Boltzmann constant, nondimensionalized

γ = 1.0 # friction
ρ = 0.7 # density
T = 0.8 # temperature
β = 1/(kB*T) # inverse temperature

α = cbrt(1/ρ) # cell length
L = α*Ns # simulation box length

boundary = CubicBoundary(L, L, L)
coords = place_atoms_on_3D_lattice(Ns, Ns, Ns, boundary)
atoms = [Atom(index = i, ϵ = ϵ, σ = σ, mass = m) for i = 1:N]
velocities = [random_velocity(m, T, kB) for i = 1:N]

cutoff = ShiftedForceCutoff(rc)
lj_w_cutoff = LennardJones(cutoff = cutoff, force_units = NoUnits, energy_units = NoUnits)
nf = CellListMapNeighborFinder(eligible = trues(N,N), unit_cell = boundary, n_steps = 1, dist_cutoff = rc)
;

### Exercise: implementing the nonequilibrium forcing

We now implement the extra forcing. The first step is to define the forcing struct, which contains the forcing profile and its magnitude.

**Task:** Complete the `NEMD_longitudinal_forcing` struct by filling in its two fields: 
- The forcing profile should be called `force_profile` (of type `F`);
- Its magnitude should be called `η` (of type `Float64`).

In [ ]:
struct NEMD_longitudinal_forcing{F}
   # fill in the fields here
end
;

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
    struct NEMD_longitudinal_forcing{F}
        force_profile::F
        η::Float64
    end
  ```
</details>

---
#### Forcing profile

Next, we define the appropriate forcing profile and create a forcing object.

**Task:** Create an object of type `NEMD_longitudinal_forcing`. This is done as follows:
- Create the forcing profile function (which should be called `sinus_forcing`);
- Create a variable which denotes its magnitude (which should be called `η`);
- Create the object of type `NEMD_longitudinal_forcing` (which should be called `forcing`).

For convenience, we recall the expression for the forcing profile defined in <a href="#sin_fp">(1)</a>:

$$
    f_y(u) = \sin\left(\frac{2\pi u}{L_y}\right).
$$

In [ ]:
# define the appropriate variables and create the object here

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
    η = 1.0 # magnitude of perturbation
    sinus_forcing = (y -> sin(2π*y/L))
    forcing = NEMD_longitudinal_forcing(sinus_forcing, η)
  ```
</details>

---
#### Force computation

At this point, we must tell Molly how to compute the forces associated with our custom force of type `NEMD_longitudinal_forcing`. To do so, we create a `Molly.forces()` method which takes in interactions of type `NEMD_longitudinal_forcing`.

**Task:** Complete the `Molly.forces()` method below. In particular, make the appropriate changes to `f` inside the function so that it outputs the correct extra force term. Recall that the force on atom $i$ is given by

$$
\text{force}_i = \eta F, \qquad F = \begin{pmatrix}f_y(q_{i2}) \\ 0 \\ 0 \end{pmatrix},
$$

where $q_{i2}$ denotes the $y$-component of the position of atom $i$.

In [ ]:
"""
    function Molly.forces(inter::NEMD_longitudinal_forcing{F}, s::System, neighbors=nothing; n_threads=Threads.nthreads()) where {F}
"""
function Molly.forces(inter::NEMD_longitudinal_forcing{F}, s::System, neighbors=nothing; n_threads=Threads.nthreads()) where {F}
    f = zero(s.velocities)
    # write your changes here
    return inter.η*f
end
;

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
function Molly.forces(inter::NEMD_longitudinal_forcing{F}, 
                        s::System, 
                        neighbors=nothing; 
                        n_threads=Threads.nthreads()) where {F}

        f = zero(s.velocities)
        f_x = view(reinterpret(reshape, Float64, f), 1, :)
        q_y = view(reinterpret(reshape, Float64, s.coords), 2, :)
        f_x .= inter.force_profile.(q_y)
        
        return inter.η*f
end
  ```
</details>

---
#### Observable of interest

We must also define a function that computes our desired response. In our case, it corresponds to the first empirical Fourier coefficient <a href="#Rk1">(7)</a>, which we recall for convenience:

$$
    R(q,p) = \frac{1}{N}\sum_{i=1}^N \frac{p_{i1}}{m}\exp\left(\frac{2i\pi q_{i2}}{L}\right).
$$

**Task:** Complete a function `fourier_response()` which outputs the result of the equation above. In particular, you must define the following
- `p_x`, the $x$-component of the momenta;
- `q_y`, the $y$-component of the positions;
- `N`, the number of particles in the system;
- `Ly`, the length of the $y$-boundary.

In [ ]:
"""
    fourier_response(s::System, args...; kwargs...)
"""
function fourier_response(s::System, args...; kwargs...)
    # write your changes here
    return dot(p_x, exp.(2im*π*q_y/Ly))/N
end
;

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
function fourier_response(s::System, args...; kwargs...)
    p_x = view(reinterpret(reshape, Float64, s.velocities), 1, :)
    q_y = view(reinterpret(reshape, Float64, s.coords), 2, :)
    Ly = s.boundary.side_lengths[2]
    N = length(s)
    
    return dot(p_x, exp.(2im*π*q_y/Ly))/N
end
  ```
</details>

---
#### Logger

We need to define a logger to track of our observable `fourier_response()` throughout the simulation.

**Task:** Define a logger named `fourier_logger` using the `GeneralObservableLogger()` constructor, as described below.

In [ ]:
"""
    GeneralObservableLogger(observable::Function, T, n_steps)

`observable` should return an object of type `T` and support the method
`observable(s::System, neighbors; n_threads::Integer)::T`.

`n_steps` denotes the logging frequency.
"""
# define fourier_logger here

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
    freq = 1 # record frequency 
    fourier_logger = GeneralObservableLogger(fourier_response, ComplexF64, freq)
  ```
</details>

---
#### System and integrator

We now define the system and choose the integrator of choice. In this case, we use the BAOAB splitting scheme.

**Task:** Complete the `System` object below by filling in the remaining necessary fields.

In [ ]:
sys = System(# add the additional fields here
                force_units=NoUnits,
                energy_units=NoUnits,
                k=kB)

simulator = LangevinSplitting(
    dt=dt,
    temperature=T,
    friction=γ,
    splitting="BAOAB",
)
;

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
sys = System(atoms=atoms,
                coords=coords,
                velocities=velocities,
                boundary=boundary,
                pairwise_inters=(lj_w_cutoff,),
                general_inters=(forcing,),
                neighbor_finder=nf,
                loggers=(fourier=fourier_logger,),
                force_units=NoUnits,
                energy_units=NoUnits,
                k=kB)

simulator = LangevinSplitting(
    dt=dt,
    temperature=T,
    friction=γ,
    splitting="BAOAB",
)  
```
</details>

---
#### Running the simulation and computing $\rho_1$

Finally, we run the simulation and obtain the observable of choice.

**Task:** Run the simulation with `simulate!()` and compute the shear viscosity. In particular, include a thermalization run (use the `run_loggers=false` kwarg for this run).

In [ ]:
# write your code here

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
Ntherm = 1000 # number of thermalization steps

simulate!(sys, simulator, Ntherm; run_loggers=false)
simulate!(sys, simulator, Nsteps)

obs = imag.(values(sys.loggers.fourier))

ρ1 = mean(obs)/η
σ_sv = ρ*(0.5/ρ1 - γ)*(L/2/pi)^2

display(σ_sv)
  ```
</details>

---
## Technique 2: Green-Kubo formula

An alternative technique for computing transport coefficients is by using the Green-Kubo formula:

$$
    \rho_1 = \int_0^{+\infty} \mathbb{E}_0\left[R(q_t,p_t)S(q_0,p_0)\right] dt,
$$

where $R$ is the observable of interest (i.e. the first empirical Fourier coefficient <a href="#Rk1">(7)</a>), and $S$ the conjugate response function (see Section 5.2.3 of [Leliévre & Stoltz, 2016](https://www.researchgate.net/publication/303596906_Partial_differential_equations_and_stochastic_methods_in_molecular_dynamics) for a precise definition).

This approach involves simulating typical equilibrium dynamics, and computing the appropriate correlation function for the transport coefficient of interest. We now discuss how to implement this approach in Molly.

### Exercise: implementing Green-Kubo

The response function used here, namely $R$, is the same one used in the nonequilibrium approach, which we defined as `fourier_response()`. We must however define the conjugate response function $S$.

**Task:** Complete the function `shear_conj_response()` so that it outputs the conjugate response function $S$, defined as

$$
    S = \beta F(q)^TM^{-1}p.
$$

In particular, you must define `N`, `q_y` and `p_x`.

In [ ]:
"""
    shear_conj_response(forcing::T) where {T}
"""
function shear_conj_response(forcing::T) where {T}
    function R(sys, args...; kwargs...)
        # write your changes here
        return dot(p_x, forcing.(q_y))/sqrt(N)*β
    end
    return R
end
;

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
function shear_conj_response(forcing::T) where {T}
    function R(sys, args...; kwargs...)
        N = length(sys)
        p_x = view(reinterpret(reshape, Float64, sys.velocities), 1, :)
        q_y = view(reinterpret(reshape, Float64, sys.coords), 2, :)
        
        return dot(p_x, forcing.(q_y))/sqrt(N)*β
    end

    return R
end
  ```
</details>

---
#### Logger

We must define the time correlation logger, which tracks the time correlation function $C(t)$ throughout the simulation. In general, $C(t)$ is of the form

$$
    C(t) =  \mathbb{E}_0[A_t\cdot B_0].
$$

In our case, it reads

$$
    C(t) =  \mathbb{E}_0[R(q_t,p_t)S(q_t,p_t)].
$$

**Task:** Define the logger, called `fourierAC_logger`, using the `TimeCorrelationLogger()` constructor, as described below.

**Remark:** The time correlation logger in Molly approximates the expectation in the correlation function over a single trajectory with block sampling (as opposed to averaging over multiple trajectories). Thus, `n_correlation` should be much smaller than `Nsteps`, while still being larger than the decorrelation time.

In [ ]:
"""
TimeCorrelationLogger(observableA::Function, observableB::Function,
                        TA::DataType, TB::DataType,
                        observable_length::Integer, n_correlation::Integer)
"""
# define fourierAC_logger here

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
    ncor = 1000 # number of correlation steps

    sin_response = shear_conj_response(sinus_forcing)
    fourierAC_logger = TimeCorrelationLogger(sin_response, fourier_response, Float64, ComplexF64, 1, ncor)
  ```
</details>

---
#### Redefining the system

Below is the system we used for the nonequilibrium approach. We must make two changes so that it is appropriate for the Green-Kubo method.

**Task:** Redefine the system after making the appropriate changes below.

In [ ]:
sys2 = System(atoms=atoms,
                coords=coords,
                velocities=velocities,
                boundary=boundary,
                pairwise_inters=(lj_w_cutoff,),
                general_inters=(forcing,),
                neighbor_finder=nf,
                loggers=(fourier=fourier_logger,),
                force_units=NoUnits,
                energy_units=NoUnits,
                k=kB)

<details>
  <summary>Click here to see answer</summary>

  ```julia
    # Only two changes were made from the nonequilibrium system, namely (i) removed general interactions field, and (ii) updated the logger
    sys2 = System(atoms=atoms,
                coords=coords,
                velocities=velocities,
                boundary=boundary,
                pairwise_inters=(lj_w_cutoff,),
                neighbor_finder=nf,
                loggers=(fourierAC=fourierAC_logger,),
                force_units=NoUnits,
                energy_units=NoUnits,
                k=kB)
  ```
</details>

---
#### Running the simulation and computing $\rho_1$

We finally run the simulation and compute the transport coefficient. Note that by time-integrating the correlation function $C(t)$, we obtain the Green-Kubo formula. This naturally suggests the following estimator for $\rho_1$:

$$
    \rho_1 \approx \Delta t\sum_{k=0}^{N_\text{corr}} \widehat{C}(k\Delta t).
$$

**Task:** Run the simulation (including a thermalization run) with `simulate!()` and compute the shear viscosity by performing a quadrature on the time correlation log values.

In [ ]:
# write your code here

<details>
  <summary>Click here to see answer</summary>
  
  ```julia
simulate!(sys2, simulator, Ntherm; run_loggers=false)
simulate!(sys2, simulator, Nsteps)

obs = imag.(values(sys2.loggers.fourierAC))

ρ1 = dt*sum(obs)
σ_sv = ρ*(0.5/ρ1 - γ)*(L/2/pi)^2

display(σ_sv)
  ```
</details>